In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import string

In [23]:
df = pd.DataFrame(dict(group = np.repeat(['T','C'], repeats=4*5)
                       ,user = np.repeat(list(map(chr, range(97, 97+8))), repeats=5)
                       ,session = np.tile(np.arange(1,6),8)
                       ,is_click = np.array([1,0,0,0,1, 1,1,0,1,np.nan, 1,0,0,np.nan,np.nan, 0,0,0,np.nan,np.nan, 0,1,0,1,1, 1,1,1,np.nan,np.nan, 0,0,1,np.nan,np.nan, 0,1,0,0,np.nan])
                       ,is_trigger = np.array([0,1,0,0,0, 1,1,1,1,np.nan, 1,0,0,np.nan,np.nan, 0,0,0,np.nan,np.nan, 0,0,1,0,0, 1,1,1,np.nan,np.nan, 0,0,0,np.nan,np.nan, 0,1,0,0,np.nan])
                      ))

In [24]:
df['is_untrigger'] = 1-df['is_trigger']
df['is_tr_clk'] = df['is_click']*df['is_trigger']
df['is_untr_clk'] = df['is_click']*(1-df['is_trigger'])

In [4]:
df_agg = df.groupby(['group','user']).agg({'is_click':'mean','is_trigger':['mean','sum'],'is_untrigger':'sum','is_tr_clk':'sum','is_untr_clk':'sum'})
df_agg.reset_index(inplace=True)
df_agg.sort_values(by='group', axis=0, ascending=False, inplace=True)
df_agg['is_tr_clk_m'] = df_agg['is_tr_clk']['sum']/df_agg['is_trigger']['sum']
df_agg['is_untr_clk_m'] = df_agg['is_untr_clk']['sum']/df_agg['is_untrigger']['sum']
df_agg

group user  is_click is_trigger      is_untrigger is_tr_clk is_untr_clk  \
                  mean       mean  sum          sum       sum         sum   
4     T    a  0.400000   0.200000  1.0          4.0       0.0         2.0   
5     T    b  0.750000   1.000000  4.0          0.0       3.0         0.0   
6     T    c  0.333333   0.333333  1.0          2.0       1.0         0.0   
7     T    d  0.000000   0.000000  0.0          3.0       0.0         0.0   
0     C    e  0.600000   0.200000  1.0          4.0       0.0         3.0   
1     C    f  1.000000   1.000000  3.0          0.0       3.0         0.0   
2     C    g  0.333333   0.000000  0.0          3.0       0.0         1.0   
3     C    h  0.250000   0.250000  1.0          3.0       1.0         0.0   

  is_tr_clk_m is_untr_clk_m  
                             
4        0.00      0.500000  
5        0.75           NaN  
6        1.00      0.000000  
7         NaN      0.000000  
0        0.00      0.750000  
1        1.00           NaN  
2         NaN      0.333333  
3        1.00      0.000000

In [5]:
df_agg.columns = ['_'.join(col).rstrip('_') for col in df_agg.columns]
df_agg.rename(columns={'is_click_mean':'X','is_trigger_mean':'TR', 'is_tr_clk_m':'TrX', 'is_untr_clk_m':'UnTrX'}, inplace=True)
df_agg['is_TR_1'] = np.array(df_agg['TR'].values==1, dtype=int)
df_agg = df_agg[['group','user','X','TR','TrX','UnTrX','is_TR_1']]
df_agg = df_agg.fillna(0)

In [6]:
Y = df_agg.loc[df_agg['group']=='C', ['UnTrX','TR','is_TR_1']].values
X = df_agg.loc[df_agg['group']=='C','X'].values
X = X-X.mean(axis=0)
Y = Y-Y.mean(axis=0)
theta = np.linalg.inv(Y.T@Y)@Y.T@X
theta

array([0.48780488, 0.31707317, 0.51219512])

In [29]:
dYm = np.mean(df_agg.loc[df_agg['group']=='T', ['UnTrX','TR','is_TR_1']].values - df_agg.loc[df_agg['group']=='C', ['UnTrX','TR','is_TR_1']].values, axis=0)
dXm = np.mean(df_agg.loc[df_agg['group']=='T','X'].values- df_agg.loc[df_agg['group']=='C','X'].values)
delta_vr = dXm - np.dot(dYm,theta)
delta_vr

-0.11046747967479675

In [35]:
X_t = df_agg.loc[df_agg['group']=='T','X'].values
X_c = df_agg.loc[df_agg['group']=='C','X'].values
X_t_dm = X_t-X_t.mean(axis=0)
X_c_dm = X_c-X_c.mean(axis=0)

Y_t = df_agg.loc[df_agg['group']=='T', ['UnTrX','TR','is_TR_1']].values
Y_c = df_agg.loc[df_agg['group']=='C', ['UnTrX','TR','is_TR_1']].values
Y_t_dm = Y_t-Y_t.mean(axis=0)
Y_c_dm = Y_c-Y_c.mean(axis=0)

n = 4

In [37]:
var_vr = np.var(X_t)/n + np.var(X_c)/n + theta.T@(np.cov(Y_t.T)/n + np.cov(Y_c.T)/n)@theta - 2*theta.T@(Y_t_dm.T@X_t_dm/(n)/n + Y_c_dm.T@X_c_dm/(n)/n)
var_vr

0.0047180471899993595

In [39]:
#### variance reduction rate
1-var_vr / (np.var(X_t)/n + np.var(X_c)/n)

0.8792984596295966

In [31]:
Z_score = delta_vr/np.sqrt(var_vr)
Z_score

-0.8041245285654302